# Concat Operator

* https://xarray.pydata.org/en/stable/user-guide/combining.html
* https://xarray.pydata.org/en/stable/user-guide/io.html#reading-multi-file-datasets

In [ ]:
import xarray as xr
import pandas as pd

In [ ]:
from pathlib import Path
data_dir = Path("/Users/pingu/data/cmip6-decadal")

In [ ]:
cmip6_files = sorted(data_dir.glob('**/*.nc'))
cmip6_files

In [ ]:
ds0 = xr.open_dataset(cmip6_files[0])
ds0

In [ ]:
ds0.realization_index

In [ ]:
datasets = [xr.open_dataset(f) for f in cmip6_files]
len(datasets)

In [ ]:
realization_values = [ds.realization_index for ds in datasets]
realization_values

In [ ]:
realization_dim = pd.Index(realization_values, name="realization")

In [ ]:
ds_concat = xr.concat(datasets, dim=realization_dim)
ds_concat

In [ ]:
ds_concat.isel(realization=0)